# First Plot Attempt
*Akira Di Sandro, 6/9/20*
<br>
First attempt at using data to plot something. Code taken mostly from Raf's tutorial notebooks.

## 1. Import python packages that we need

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import xarray as xr
xr.set_options(display_style='html')
import intake
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

## 2. Load CMIP data to find CM4 dataset
I want this dataset to specifically have velocity data so I can plot these.

In [3]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

,unique
activity_id,15
institution_id,33
source_id,73
experiment_id,103
member_id,159
table_id,29
variable_id,370
grid_label,10
zstore,269644
dcpp_init_year,60


Refine Search: specifically changed the variable_id from 'tos' (sea surface temperature) to 'uo' (eastward sea water veolicty) and changed experiment_id from both 'historical and 'ssp585' to just 'historical'.

In [36]:
dog = col.search(source_id='GFDL-CM4', experiment_id='historical', table_id='Omon', variable_id=['uo','vo','wo'], grid_label='gn')
#variable_id: uo=eastward_sea_water_velocity, vo=northward_sea_water_velocity, wo=upward_sea_water_velocity
dog.df
#called dog instead of cat just because I was using Raf's code in here to help me debug & wanted my code to be different from his

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
0,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,Omon,uo,gn,gs://cmip6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...,NaN
1,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,Omon,vo,gn,gs://cmip6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...,NaN


In [38]:
dset_dict = dog.to_dataset_dict(zarr_kwargs={'consolidated': True})
list(dset_dict.keys())

Dataset(s):   0%|                                        | 0/1 [00:00<?, ?it/s]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


Dataset(s): 100%|████████████████████████████████| 1/1 [00:04<00:00,  4.06s/it]


['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']

In [41]:
CM4_hist = dset_dict['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']
CM4_hist

<xarray.Dataset>
Dimensions:    (bnds: 2, lev: 35, member_id: 1, time: 1980, x: 1440, y: 1080)
Coordinates:
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(35, 2), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * bnds       (bnds) float64 1.0 2.0
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * member_id  (member_id) <U8 'r1i1p1f1'
Data variables:
    uo         (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 35, 1080, 1440), meta=np.ndarray>
    vo         (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 35, 1080, 1440), meta=np.ndarray>
Attributes:
    experiment_id:           historical
    realm:                   ocean
    grid:                    ocean data on native grid tripolar - nominal 0.2...
    experiment:              historical
    forcing_index:           1
    frequency:               mon
    mip_era:                 CMIP6
    activity_id:             CMIP
    sub_experiment_id:       none
    variant_info:            N/A
    branch_time_in_child:    0.0
    history:                 File was processed by fremetar (GFDL analog of C...
    status:                  2019-08-06;created;by nhn2@columbia.edu
    license:                 CMIP6 model data produced by NOAA-GFDL is licens...
    source_id:               GFDL-CM4
    references:              see further_info_url attribute
    product:                 model-output
    variant_label:           r1i1p1f1
    data_specs_version:      01.00.27
    branch_method:           standard
    source_type:             AOGCM
    source:                  GFDL-CM4 (2018): \naerosol: interactive\natmos: ...
    initialization_index:    1
    realization_index:       1
    parent_experiment_id:    piControl
    physics_index:           1
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.G...
    contact:                 gfdl.climate.model.info@noaa.gov
    nominal_resolution:      25 km
    comment:                 <null ref>
    grid_label:              gn
    parent_source_id:        GFDL-CM4
    institution_id:          NOAA-GFDL
    title:                   NOAA GFDL GFDL-CM4 model output prepared for CMI...
    Conventions:             CF-1.7 CMIP-6.0 UGRID-1.0
    parent_activity_id:      CMIP
    table_id:                Omon
    parent_mip_era:          CMIP6
    parent_variant_label:    r1i1p1f1
    parent_time_units:       days since 0001-1-1
    institution:             National Oceanic and Atmospheric Administration,...
    tracking_id:             hdl:21.14100/d0ebcd36-d2cd-442e-a8d9-8f7a27168a2...
    sub_experiment:          none
    branch_time_in_parent:   36500.0
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn

In [42]:
uo_2012 = CM4_hist['uo'].sel(time='2012').mean(dim='time')
uo_2012

<xarray.DataArray 'uo' (member_id: 1, lev: 35, y: 1080, x: 1440)>
dask.array<mean_agg-aggregate, shape=(1, 35, 1080, 1440), dtype=float32, chunksize=(1, 35, 1080, 1440), chunktype=numpy.ndarray>
Coordinates:
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * member_id  (member_id) <U8 'r1i1p1f1'